***Численное дифференцирование***

In [2]:
from IPython.display import Latex
from math import sin,cos, atan, pi
import pandas as pd
from matplotlib import pyplot as plt

*Функция для численного дифференцирования*

In [12]:
%%latex
\begin{align}
f(x) \space = \space sin^2(x), \space x\in [-1.5,1.5]
\end{align}

<IPython.core.display.Latex object>

In [8]:
def f(X):
    return sin(x) ** 2

Используя правую и центральную разности вычислить производную в узлах сетки.

In [9]:
def get_interpolation_net(a,b, count):
    return [a + (b - a) * i / count for i in range(count+1)]

*Производная заданной функции взятая аналитически*

In [13]:
%%latex
\begin{align}
\frac{\mathrm{d} f}{\mathrm{d} x} \space = \space 2sin(x)cos(x)
\end{align}

<IPython.core.display.Latex object>

In [14]:
def analytic_derivative(x):
    assert isinstance(x, float)
    return 2*sin(x)*cos(x)